### <b style="color: #abcdef">04.객체 검출과 이미지 세분화 - 객체검출(Detection)</b>

> #### <b style="color: #58a491">detection</b>
> - 컨볼루션 레이어는 위치정보를 추출함
> - 컨볼루션 스택에 요소추가하여 네트워크 훈련

> #### <b style="color: #f86461">YOLO</b>
> - 컨볼루션 백본 끝에 요소 추가  
> ---
> <b style="color: #53a593">YOLO 격자</b>
> - 이미지 분할 N * M 
> - 중심좌표, 너비, 높이, 확신도 => (x, y, w, h, C)  
> - 객체 검출 헤드 5(x, y, w, h, C)  + class수 채널  
> ---
> <b style="color: #53a593">손실</b>
> - 객체존재손실: (1-C)**2
> - 객체부재손실: (0-C)\**2 = C**2
> - 객체분류손실: cross_entropy
> - 경계박스손실: (x-x^)**2 + (y-y^)**2 + (sqrt(w) - sqrt(w^)) ** 2 + (sqrt(h) - sqrt(h^))**2  
> ---
> <b style="color: #53a593">손실가중치</b> - 객체 부재의 손실이 압도적으로 많아 객체 존재 손실이 묻힌다
> - 객체존재손실: 1
> - 객체부재손실: 0.5
> - 객체분류손실: 1
> - 경계박스손실: 5  
> <b style="color: #53a593">한계</b> - 격자셀당 하나의 객체만 인식 가능  

> #### <b style="color: #f86461">레티나넷(RetinaNet)</b>
> ---
> <b style="color: #53a593">FPN</b>
> - 저수준의 공간정보 + CNN의 고수준 의미정보 (상위 특징맵 + 하위 특징맵 업샘플링 + 검출헤드)  
> ---
>   <b style="color: #53a593">앵커박스</b>
> - 종횡비: 2:1, 1:1, 1:2
> - 크기: 2\*\*0, 2\*\*(1/3), 2**(2/3) (대략 1, 1.3, 1.6)
> - 스케일: P3 ~ P7 (2**n => 8, 16, 32, 64, 128, 256, 512)
> - 앵커베이스 크기 4*2**n => 32, 64, 128, 256, 512 <b style="color: #ff5555">뭔소린지 전혀 모르겠다</b>
> - 앵커박스 간격 8, 16, 32, 64, 128, 256, 512
> - 앵커박스 크기: (32, 32) ~ (812, 1624) <== (512 * 2**(2/3), 512 * 2**(2/3) * 2)  
> ---
> <b style="color: #53a593">검출손실</b>  
> - ![anchor-iou-matrix](./resources/anchor-iou-matrix.png)  
> ---
> <b style="color: #53a593">아키텍처</b>  
> - K = 클래스 수, B = 박스형태 수(3가지 비율, 3가지 크기 = 9)  
> - 클래스 예측헤드: 클래스 수만큼 모든 앵커에 대한 확률 예측 (K * 9)  
> - 검출헤드: B * 4개 박스델타 예측 (x, y, w, h) => 9 * 4 = 36  
> ---
> <b style="color: #53a593">초점손실(focal loss)</b>  
> - **뭐라는거야...**  
> - 배경 앵커가 갖는 손실값 **총합**이 너무 커서(많으니까) 기여도를 많이 줄여서 그 영향을 줄이는 개념  
> - CE(y, p) = -y*log(p) - (1-y)*log(1-p)
> - FL(y, p) = -y\*log(p)*(1-p)**γ - (1-y)*log(1-p)*p\**γ  
> ---
> <b style="color: #53a593">부드러운 L1손실</b>  
> - L1 = 절대손실, L2 = 제곱 손실(squared) -- mean squared error? --
> - L1은 미분이 안되고 L2는 손실이 크면 지나치게 커짐 -> δ 지정하여 구간별로 앞부분 L2, 뒷부분 L1
> - ![anchor-iou-matrix](./resources/smooth-l1.png)
> - ![anchor-iou-matrix](./resources/smooth-l1-v2.png)
> ---
> <b style="color: #53a593">NMS(Non Maximum Supression)</b>  
> - IOU가 임계값 이상인 박스 그룹에서 확신도가 최대인 박스만 남긴다  
> - 근접한 객체가 삭제될 소지가 있다(IOU가 생기니까)
> - Soft-NMS 를 사용해서 박스를 없애지 않고 C값을 낮춰 해결 -> 검출목록은 σ값이 결정(뭔지는 잘..)
> ---
> <b style="color: #53a593">그밖의 고려사항...</b>  
> - <b style="color: #f09433">레티나넷 얘기하다가 뭔 고려사항이여</b>
> - 데이터 증강 하란다

### <b style="color: #abcdef">04.객체 검출과 이미지 세분화 - 세분화(Segmentation)</b>
- instance segmentation: 각 객체의 모양을 제공하는 픽셀 마스크 추가
- semantic segmentation: 이미지의 모든 픽셀을 범주로 분류

> #### <b style="color: #f86461">Mask R-CNN</b>
> ---
> <b style="color: #53a593">RPN(Region Proposal Network</b>
> - 2개의 망을 통과(배경/객체, 앵커박스) + 객체구분  
> - 박스 손실: 후버 => NMS => 관심영역(ROI: regions of interest)  
> - 클래스: 이진교차 엔트로피  
> - 가능성 높은 객체의 관심영역을 대략 계산하는게 목표  
> ---
> <b style="color: #53a593">R-CNN</b>  
> - 이미지 -> **백본** -> Feature Map -> boxes => **백본** -> 분류  
> - 2차 과정은 개별 박스가 아니라 박스가 표시된 Feature Map을 입력한다  
> - FPN에서 특징맵 선택? 관련성 높은 FPN 수준?  
> - ![anchor-iou-matrix](./resources/FPN_관련성.png)  
> - <b style="color: red">???</b>
> - 5개 FPN 수준 P2, p3, p4, p5, p6 (Pn은 w, h가 2**n배 작은 feature map)
> - 앵커박스 32, 64, 128, 256, 512
> - n0 = 4
> - **뭐라는거야!!!**
>  
> 80 * 160 => p3 ??? p3이면 224/(2**3) => 224/8 => 28 인데???
> - 80 * 160 = 1280, 224 * 224 = 50176, 50176 / 8 = 6272
> - 224 * 224 / (2**5) = 1568
> - <b style="color: red; font-size: 20px;">???</b>
> ---
> <b style="color: #53a593">관심영역 리샘플링(관심 영역 정렬)</b>  
> - 반올림 오차가 성능에 부정적
> - bilinear interpolation
> - **계산이 정확해야 한다**
> ---
> <b style="color: #53a593">전치컨볼루션(transposed convolution)</b>  
> - 디코더 레이어로 사용  
> - 바둑판 현상이 발생함 -> up convolution으로 해결  
> ---
> <b style="color: #53a593">인스턴스 세분화(segmentation) 헤더</b>  
> - 전치 컨볼루션 사용  
> - K개의 마스크 예측  
> ---
> #### <b style="color: #f86461">U-Net</b>
> U-Net은 U-Net